Import the analysis package.

In [ ]:
import cosmicfish as cf 

Specify the values of your fiducial cosmology.

In [ ]:
A_s = 2.2321e-9
n_s = 0.96659
omega_b = 0.02226
omega_cdm = 0.11271
tau_reio = 0.059888
h = 0.70148

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc

In [ ]:
import os

for _, e, _ in os.walk("/Users/nicholasdeporzio/Desktop/test/data/"): 
    print(e)

In [ ]:
for e in next(os.walk("/Users/nicholasdeporzio/Desktop/test/data/"))[1]: 
    print(int(e))

In [ ]:
s1 = cf.spectrum("~/Desktop/test/data/1/")

In [ ]:
s1.rawdata.head()

In [ ]:
s1.rawdata['d_b'][0]

In [ ]:
s1.rawdata['d_cdm'][1]